In [13]:
from torch.utils.data import DataLoader
from torchvision import transforms, models, datasets
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import matplotlib.pyplot as plt

In [14]:
device = torch.device("cuda")

In [4]:
dataloader = 1

In [1]:
!pip install augmentor

In [7]:
from skorch import NeuralNetClassifier
from sklearn.model_selection import cross_val_score


In [15]:
data_dir = r'D:\majorProject\data\trashnet'

In [16]:
def load_split_train_test(datadir, valid_size = .2):
    train_transforms = transforms.Compose([transforms.Resize(224),
                                           transforms.ToTensor(),
                                           # transforms.RandomAffine([0,180]),
                                           # transforms.RandomVerticalFlip(),
                                           # transforms.RandomHorizontalFlip()
                                       ])
    test_transforms = transforms.Compose([transforms.Resize(224),
                                          transforms.ToTensor(),
                                        # transforms.RandomAffine([0,180]),
                                        #    transforms.RandomVerticalFlip(),
                                        #    transforms.RandomHorizontalFlip()
                                      ])
    train_data = datasets.ImageFolder(datadir,       
                    transform=train_transforms)
    test_data = datasets.ImageFolder(datadir,
                    transform=test_transforms)
    num_train = len(train_data)
    indices = list(range(num_train))
    split = int(np.floor(valid_size * num_train))
    np.random.shuffle(indices)
    from torch.utils.data.sampler import SubsetRandomSampler
    train_idx, test_idx = indices[split:], indices[:split]
    train_sampler = SubsetRandomSampler(train_idx)
    test_sampler = SubsetRandomSampler(test_idx)
    trainloader = torch.utils.data.DataLoader(train_data,
                   sampler=train_sampler, batch_size=16)
    testloader = torch.utils.data.DataLoader(test_data,
                   sampler=test_sampler, batch_size=16)
    return trainloader, testloader

trainloader, testloader = load_split_train_test(data_dir, .2)


In [10]:
model_conv = models.resnet18(pretrained=True)

for param in model_conv.parameters():
    param.requires_grad = False

num_ftrs = model_conv.fc.in_features
model_conv.fc = nn.Linear(num_ftrs, 384)
model_conv.relu1 = nn.ReLU()
model_conv.dr1 = nn.Dropout(p=0.5)
model_conv.fc1 = nn.Linear(384, 128)
model_conv.relu2 = nn.ReLU()
model_conv.dr2 = nn.Dropout(p=0.3)
model_conv.fc2 = nn.Linear(128, 64)
model_conv.relu3 = nn.ReLU()
model_conv.dr3 = nn.Dropout(p=0.2)
model_conv.fc3 = nn.Linear(64,6)
model_conv.soft = nn.Softmax(dim=1)

model = model_conv.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that only parameters of final layer are being optimized as
# opposed to before.
optimizer = optim.SGD(model.fc.parameters(),lr =0.1, momentum=0.5 )

# # Decay LR by a factor of 0.1 every 7 epochs
# exp_lr_scheduler = lr_scheduler.StepLR(optimizer_conv, step_size=7, gamma=0.1)



In [11]:
# num_ftrs = model_conv.fc.in_features
# print(num_ftrs)

In [ ]:
epochs = 5
steps = 0
running_loss = 0
print_every = 10
train_losses, test_losses = [], []
for epoch in range(epochs):
    for inputs, labels in trainloader:
        steps += 1
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        logps = model.forward(inputs)
        loss = criterion(logps, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        
        if steps % print_every == 0:
            test_loss = 0
            accuracy = 0
            model.eval()
            with torch.no_grad():
                for inputs, labels in testloader:
                    inputs, labels = inputs.to(device), labels.to(device)
                    logps = model.forward(inputs)
                    batch_loss = criterion(logps, labels)
                    test_loss += batch_loss.item()
                    
                    ps = torch.exp(logps)
                    top_p, top_class = ps.topk(1, dim=1)
                    equals = top_class == labels.view(*top_class.shape)
                    accuracy += torch.mean(equals.type(torch.FloatTensor)).item()
            train_losses.append(running_loss/len(trainloader))
            test_losses.append(test_loss/len(testloader))                    
            print(f"Epoch {epoch+1}/{epochs}.. "
                  f"Train loss: {running_loss/print_every:.3f}.. "
                  f"Test loss: {test_loss/len(testloader):.3f}.. "
                  f"Test accuracy: {accuracy/len(testloader):.3f}")
            running_loss = 0
            model.train()
            



Epoch 1/5.. Train loss: 0.488.. Test loss: 0.532.. Test accuracy: 0.816
Epoch 1/5.. Train loss: 0.573.. Test loss: 0.473.. Test accuracy: 0.834
Epoch 1/5.. Train loss: 0.568.. Test loss: 0.469.. Test accuracy: 0.834
Epoch 1/5.. Train loss: 0.596.. Test loss: 0.484.. Test accuracy: 0.848
Epoch 1/5.. Train loss: 0.533.. Test loss: 0.454.. Test accuracy: 0.838
Epoch 1/5.. Train loss: 0.602.. Test loss: 0.442.. Test accuracy: 0.844
Epoch 1/5.. Train loss: 0.525.. Test loss: 0.477.. Test accuracy: 0.844
Epoch 1/5.. Train loss: 0.469.. Test loss: 0.450.. Test accuracy: 0.837
Epoch 1/5.. Train loss: 0.521.. Test loss: 0.443.. Test accuracy: 0.846
Epoch 1/5.. Train loss: 0.594.. Test loss: 0.438.. Test accuracy: 0.851
Epoch 1/5.. Train loss: 0.537.. Test loss: 0.454.. Test accuracy: 0.852
Epoch 1/5.. Train loss: 0.562.. Test loss: 0.472.. Test accuracy: 0.829
Epoch 2/5.. Train loss: 0.465.. Test loss: 0.445.. Test accuracy: 0.858
Epoch 2/5.. Train loss: 0.487.. Test loss: 0.445.. Test accuracy

In [17]:

torch.save(model.state_dict(), 'resnet50_statedict.pth')

In [18]:
# resnet 50 and simple block

model = models.resnet50(pretrained=True)
print(model.layer4)

for param in model.parameters():
    param.requires_grad = False

model.layer4.requires_grad = True


model.fc = nn.Sequential(nn.Linear(2048, 512),
                                 nn.ReLU(),
                                 nn.Dropout(0.2),
                                 nn.Linear(512, 6),
                                 nn.LogSoftmax(dim=1))
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.0001)
model.load_state_dict(torch.load('resnet50_statedict.pth'))
model.to(device)


Sequential(
  (0): Bottleneck(
    (conv1): Conv2d(1024, 512, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv2): Conv2d(512, 512, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (bn2): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv3): Conv2d(512, 2048, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn3): BatchNorm2d(2048, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace)
    (downsample): Sequential(
      (0): Conv2d(1024, 2048, kernel_size=(1, 1), stride=(2, 2), bias=False)
      (1): BatchNorm2d(2048, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (1): Bottleneck(
    (conv1): Conv2d(2048, 512, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv2): Conv2

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=F

In [ ]:
plt.plot(train_losses, label='Training loss')
plt.plot(test_losses, label='Validation loss')
plt.legend(frameon=False)
plt.show()
